### 1. 呼叫LLM

In [ ]:
import dotenv
import os
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("GEMINI_API_KEY")
os.environ["OPENAI_BASE_URL"] = os.getenv("GEMINI_BASE_URL")

llm = ChatOpenAI(model=os.getenv("GEMINI_MODEL"))

response = llm.invoke("你好")
print(response)

### 2.使用提示詞模板

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一個專業的技術文檔編寫者"),
    ("user", "{input}"),
])

chain = prompt | llm
message = chain.invoke({"input": "大模型中的LangChain是什麼?"})
print(message)

### 3.使用輸出解析器

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

llm = ChatOpenAI(model=os.getenv("GEMINI_MODEL"))

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一個專業的技術文檔編寫者"),
    ("user", "{input}"),
])

# 輸出解析器，要求回傳結果是JSON
output_parser = JsonOutputParser()

# 串起來
chain = prompt | llm | output_parser
message = chain.invoke({"input": "LangChain是什麼?用JSON格式回覆，問題用question，回答用answer"})
print(message)

### 4.使用向量儲存

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    web_path="https://forum.gamer.com.tw/C.php?bsn=74934&snA=10860",
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(id="cf83311"))
)

docs = loader.load()


from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001",
    api_key=os.getenv("GEMINI_API_KEY"),
)

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 使用分割器分割文檔
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)
print(len(documents))

# 儲存向量
vector = FAISS.from_documents(documents, embeddings)

### 5.使用RAG

In [ ]:
from langchain_core.prompts import PromptTemplate

# 設定搜尋器，k是每次找最相似的X段文字
retriever = vector.as_retriever(
    search_kwargs={"k": 3}
)

prompt_template = """
你是一個問答機器人。
你的任務是根據下述給定的已知訊息回答使用者問題。
確保你的回覆完全根據下述已知訊息，不要編造答案。
如果下述已知訊息不足以回答使用者的問題，請直接回覆"我無法回答您的問題"。

已知訊息:
{info}

使用者問題:
{question}

請使用繁體中文回答使用者問題。
"""
# 建立提示詞模板
template = PromptTemplate.from_template(prompt_template)
# prompt = template.format(info=docs, question="在鳴潮中，傷害的乘區一共有哪幾種?")
prompt = template.format(info=docs, question="白值跟黃值分別是什麼意思?")

response = llm.invoke(prompt)
print(response.content)


### 6.使用Agent

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools.retriever import create_retriever_tool
from langchain.tools import tool
from langchain_classic.agents import create_tool_calling_agent

from langchain_classic.agents import AgentExecutor


llm = ChatGoogleGenerativeAI(
    model=os.getenv("GEMINI_MODEL"),
    temperature=0
)

@tool
def multiply(a: int, b: int) -> int:
    """兩個數字相乘"""
    return a * b

tools = [multiply]

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一個會使用工具的聰明助理"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True   # 很重要，方便你 debug
)
result = agent_executor.invoke({
    "input": "請幫我算 7 乘以 8"
})

print(result["output"])